In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [5]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRUPP6U/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRUPP6U/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRUPP6U/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [9]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer= 'adam', loss= tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics= ['Accuracy'])

In [22]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs= 20, validation_data= validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 93ms/step - loss: 6.1275e-04 - Accuracy: 1.0000 - val_loss: 0.8968 - val_Accuracy: 0.8531
Epoch 2/20
30/30 [==============================] - 3s 91ms/step - loss: 5.9610e-04 - Accuracy: 1.0000 - val_loss: 0.9008 - val_Accuracy: 0.8531
Epoch 3/20
30/30 [==============================] - 2s 70ms/step - loss: 5.7461e-04 - Accuracy: 1.0000 - val_loss: 0.9050 - val_Accuracy: 0.8531
Epoch 4/20
30/30 [==============================] - 3s 91ms/step - loss: 5.5906e-04 - Accuracy: 1.0000 - val_loss: 0.9085 - val_Accuracy: 0.8531
Epoch 5/20
30/30 [==============================] - 4s 117ms/step - loss: 5.4021e-04 - Accuracy: 1.0000 - val_loss: 0.9123 - val_Accuracy: 0.8529
Epoch 6/20
30/30 [==============================] - 3s 91ms/step - loss: 5.2522e-04 - Accuracy: 1.0000 - val_loss: 0.9162 - val_Accuracy: 0.8529
Epoch 7/20
30/30 [==============================] - 3s 90ms/step - loss: 5.0921e-04 - Accuracy: 1.0000 - val_loss: 0.9200 - val_A

In [23]:
results= model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))


49/49 - 2s - loss: 1.0976 - Accuracy: 0.8345 - 2s/epoch - 37ms/step
loss: 1.098
Accuracy: 0.835
